# Data preparation

In [5]:
import os
import tensorflow as tf
import numpy as np
import pickle
from glob import glob

pkl_data_dir = os.path.join(os.getcwd(), 'data', 'xy_data_pkl')
tf_record_data_dir = os.path.join(os.getcwd(), 'data', 'xy_data_tfrecord')

# Create synthetic data

In [4]:
for i in range(100):
    d = {
        'start': np.random.rand(1,7),
        'goal': np.random.rand(1,7),
        'start_cartesian': np.random.rand(1,7),
        'goal_cartesian': np.random.rand(1,7),
        'voxels': np.random.rand(2048,3),
        'planned_result': np.random.rand(5,7),
        'planned_result_cartesian': np.random.rand(5,7),
        'planned_result_cartesian_interpolated': np.random.rand(11,7),
    }
    with open(os.path.join(pkl_data_dir, 'data_{}.pkl'.format(i)), 'wb') as f:
        pickle.dump(d, f)

# Translate pkl data to tfrecord

In [5]:
pkl_data_filenames = [fn for fn in glob(os.path.join(pkl_data_dir, '*.pkl'))]

def _bytes_feature(value):
    value = value.tostring()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def data_exampe(d):
    feature = {
        'start_cartesian': _bytes_feature(d['start_cartesian']),
        'start_cartesian_shape': _bytes_feature(np.array(d['start_cartesian'].shape)),
        'goal_cartesian': _bytes_feature(d['goal_cartesian']),
        'goal_cartesian_shape': _bytes_feature(np.array(d['goal_cartesian'].shape)),
        'voxels': _bytes_feature(d['voxels']),
        'voxels_shape': _bytes_feature(np.array(d['voxels'].shape)),
        'planned_result_cartesian_interpolated': _bytes_feature(d['planned_result_cartesian_interpolated']),
        'planned_result_cartesian_interpolated_shape': _bytes_feature(np.array(d['planned_result_cartesian_interpolated'].shape))
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

with tf.io.TFRecordWriter(os.path.join(tf_record_data_dir, 'data.tfrecord')) as writer: # python_io is io in tf2
    for fn, i in zip(pkl_data_filenames, range(len(pkl_data_filenames))):
        d = pickle.load(open(fn, 'rb'))
        tf_example = data_exampe(d)
        writer.write(tf_example.SerializeToString())


# Read TFRecord data

In [3]:
tf_record_data_dir = os.path.join(os.getcwd(), 'data', 'xy_data_tfrecord')
#tfrecord_data_filename = [fn for fn in glob(os.path.join(pkl_data_dir, '*.pkl'))]
tfrecord_data_filename = os.path.join(tf_record_data_dir, 'data.tfrecord')

print(tfrecord_data_filename)
raw_dataset = tf.data.TFRecordDataset(tfrecord_data_filename)#.batch(3) 
print(raw_dataset)


feature_description = {
    'start_cartesian': tf.io.FixedLenFeature([], tf.string),
    'start_cartesian_shape': tf.io.FixedLenFeature([], tf.string),
    'goal_cartesian': tf.io.FixedLenFeature([], tf.string),
    'goal_cartesian_shape': tf.io.FixedLenFeature([], tf.string),
    'voxels': tf.io.FixedLenFeature([], tf.string),
    'voxels_shape': tf.io.FixedLenFeature([], tf.string),
    'planned_result_cartesian_interpolated': tf.io.FixedLenFeature([], tf.string),
    'planned_result_cartesian_interpolated_shape': tf.io.FixedLenFeature([], tf.string),
}

def _parse_function(example_proto):
    return tf.io.parse_single_example(example_proto, feature_description)

def _parse_function2(example_proto):
    sample = tf.io.parse_single_example(example_proto, feature_description)
    return sample

parsed_dataset = raw_dataset.map(_parse_function2)


for _ in range(5):
    parsed_dataset.shuffle(buffer_size=50).batch(3)
    itr = parsed_dataset.__iter__()
    datapoint = itr.next()
    #print(datapoint)
    print(np.frombuffer(datapoint['start_cartesian'].numpy(), dtype=np.float64))#     for datapoint, i in zip(parsed_dataset, range(100)):
#         if i == 0:
#            print(np.frombuffer(datapoint['start_cartesian'].numpy(), dtype=np.float64))
#     #start_cartesian = np.frombuffer(datapoint['start_cartesian'].numpy(), dtype=np.float64)
    #start_cartesian_shape = np.frombuffer(datapoint['start_cartesian_shape'].numpy(), dtype=np.uint64)
    #print(start_cartesian.reshape(start_cartesian_shape))

/tf/pointnet2/data/xy_data_tfrecord/data.tfrecord
<TFRecordDatasetV2 shapes: (), types: tf.string>
[0.14059019 0.24450589 0.99172468 0.20344827 0.78783066 0.07847824
 0.5018477 ]
[0.14059019 0.24450589 0.99172468 0.20344827 0.78783066 0.07847824
 0.5018477 ]
[0.14059019 0.24450589 0.99172468 0.20344827 0.78783066 0.07847824
 0.5018477 ]
[0.14059019 0.24450589 0.99172468 0.20344827 0.78783066 0.07847824
 0.5018477 ]
[0.14059019 0.24450589 0.99172468 0.20344827 0.78783066 0.07847824
 0.5018477 ]


# LSTM Keras Test

In [1]:
import tensorflow.keras as k
from IPython.display import SVG

encoder_inputs = k.Input(shape=(None, 10))
encoder = k.layers.LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = k.Input(shape=(None, 6))
decoder = k.layers.LSTM(50, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder(decoder_inputs, initial_state=encoder_states)
decoder_outputs = k.layers.Dense(6, activation='softmax')(decoder_outputs)

model = k.Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 10)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 6)]    0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 50), (None,  12200       input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 50), ( 11400       input_2[0][0]                    
                                                                 lstm[0][1]                   

# Sandbox

ValueError: cannot reshape array of size 34 into shape (3,newaxis,6)